In [1]:
import requests
import random
import pandas as pd
import time
from tqdm import tqdm
from my_utils import *
import random

In [2]:
 riot_api_key='RGAPI-4445a220-fdde-47a7-a07f-258499614db7'

In [3]:
def get_puu_id(summoner_id,riot_api_key):
    url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={riot_api_key}'
    res = requests.get(url).json()
    puu_id = res['puuid']
    return puu_id

In [4]:
def get_match_ids(puu_id,riot_api_key,page=20):
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?type=ranked&start=0&count={page}&api_key={riot_api_key}'
    res = requests.get(url).json()
    return res

In [5]:
def get_matches_timelines(match_ids,riot_api_key):
    match_info_list = []
    for match_id in tqdm(match_ids):
        url1 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={riot_api_key}'
        url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={riot_api_key}'
        res1 = requests.get(url1).json()
        res2 = requests.get(url2).json()
        match_info_list.append([match_id,res1,res2])
    return match_info_list

In [6]:
def get_rawdata(tier):
    # 원하는 티어를 넣으면 해당 디비전에 속한 사람들 특정 몇명만 뽑는 로직
    summoner_id_list = []
    lst = []
    puu_id_list= []
    divisions = ['I','II','III','IV']
    page = random.randrange(1,51)

    print('get SummonerId')
    for division in tqdm(divisions):
        url = f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={riot_api_key}'
        res = requests.get(url).json()
        #res에 있는 데이터들 중에 랜덤으로 3개를 출력해라. 
        lst += random.sample(res,3)
    summoner_id_list = list(map(lambda x:x['summonerId'],lst))
    
    
    # 위에서 뽑은 사람들의 puu_id를 가져오는 로직
    puu_id_list=[]
    print('get PUUID......')
    for summoner_id in tqdm(summoner_id_list):
        puu_id = get_puu_id(summoner_id,riot_api_key)
        puu_id_list.append(puu_id)
    # puu_id들을 통해 match_id들을 뽑는 로직
    
    print('get match id.....')
    match_id_lst = []
    for p in tqdm(puu_id_list):
        match_ids = get_match_ids(p,riot_api_key,2)
        match_id_lst.extend(match_ids)
        
    # match_id를 통해 match_id / matches / timelines가 들어있는 데이터 프레임 만들기 
    print('get maches&timeline....')
    df_create = get_matches_timelines(match_id_lst, riot_api_key)
    df = pd.DataFrame(df_create,columns = ['match_id','matches','timeline'])
    
    # get_maches_time(ines)

    return df

In [7]:
df = get_rawdata('SILVER')

get SummonerId


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.36it/s]


get PUUID......


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.60it/s]


get match id.....


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.60it/s]


get maches&timeline....


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:37<00:00,  1.57s/it]


In [8]:
df

,match_id,matches,timeline
0,KR_7042551213,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_7042479127,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_7040836861,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_7040796391,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_7042494950,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
5,KR_7040191993,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
6,KR_7038241577,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
7,KR_7038207672,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
8,KR_7018414418,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
9,KR_7009120142,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


In [9]:
df.matches.iloc[0]['info']['teams'][0]['bans'] # blue 팀의 ban

[{'championId': 35, 'pickTurn': 1},
 {'championId': 53, 'pickTurn': 2},
 {'championId': 56, 'pickTurn': 3},
 {'championId': 266, 'pickTurn': 4},
 {'championId': 777, 'pickTurn': 5}]

## blue_ban/red_ban => 두 개를 하나의 리스트에 담기(중복이 들어왔을 경우 중복 제거)

In [10]:
df.matches.iloc[0]['info']['teams'][0]['bans']

[{'championId': 35, 'pickTurn': 1},
 {'championId': 53, 'pickTurn': 2},
 {'championId': 56, 'pickTurn': 3},
 {'championId': 266, 'pickTurn': 4},
 {'championId': 777, 'pickTurn': 5}]

In [11]:
blue_ban = list(map(lambda x:str(x['championId']), df.matches.iloc[0]['info']['teams'][0]['bans']))

In [12]:
red_ban = list(map(lambda x:str(x['championId']), df.matches.iloc[0]['info']['teams'][1]['bans']))

In [13]:
blue_ban

['35', '53', '56', '266', '777']

In [14]:
red_ban

['111', '268', '122', '154', '53']

In [15]:
ban_list = list(set(blue_ban+red_ban))

In [16]:
ban_list

['266', '268', '56', '154', '53', '111', '122', '777', '35']

In [17]:
# 특정 문자열을 리스트 요소마다 하나씩 붙여서 나오게
result = '|'.join(ban_list)
# 얘를 하는 이유는 sql에 리스트가 들어가면 str로 들어가서 사용하기가 힘들기 때문

In [18]:
result

'266|268|56|154|53|111|122|777|35'

# 킬/데스/어시 한사람의 목록을 데이터 프레임에 넣기

In [32]:
df.iloc[0].timeline['info']['frames'][28]['events'] # 게임시작 후 딱 28분일 때 이벤트들

[{'creatorId': 9,
  'timestamp': 1625017,
  'type': 'WARD_PLACED',
  'wardType': 'BLUE_TRINKET'},
 {'itemId': 3070,
  'participantId': 8,
  'timestamp': 1630579,
  'type': 'ITEM_DESTROYED'},
 {'itemId': 3802,
  'participantId': 8,
  'timestamp': 1630579,
  'type': 'ITEM_DESTROYED'},
 {'itemId': 3003,
  'participantId': 8,
  'timestamp': 1630579,
  'type': 'ITEM_PURCHASED'},
 {'assistingParticipantIds': [4, 5],
  'bounty': 0,
  'killerId': 2,
  'killerTeamId': 100,
  'monsterSubType': 'CHEMTECH_DRAGON',
  'monsterType': 'DRAGON',
  'position': {'x': 9890, 'y': 4567},
  'timestamp': 1630911,
  'type': 'ELITE_MONSTER_KILL'},
 {'itemId': 3003,
  'participantId': 8,
  'timestamp': 1630978,
  'type': 'ITEM_DESTROYED'},
 {'itemId': 2420,
  'participantId': 8,
  'timestamp': 1633315,
  'type': 'ITEM_PURCHASED'},
 {'killerId': 10,
  'timestamp': 1633482,
  'type': 'WARD_KILL',
  'wardType': 'YELLOW_TRINKET'},
 {'creatorId': 10,
  'timestamp': 1633482,
  'type': 'WARD_PLACED',
  'wardType': 'UND

In [20]:
# [{},{},{}] 중에서 내가 원하는 요소를 뽑아내는거 = filter

In [38]:
kill_list = list(filter(lambda x: x if x['type'] == 'CHAMPION_KILL' else '' ,df.iloc[0].timeline['info']['frames'][25]['events']))

In [39]:
kill_list # 여기서 killer id를 for문을 안쓰고 넣어보자

[{'assistingParticipantIds': [7],
  'bounty': 300,
  'killStreakLength': 0,
  'killerId': 8,
  'position': {'x': 10349, 'y': 6487},
  'shutdownBounty': 0,
  'timestamp': 1458237,
  'type': 'CHAMPION_KILL',
  'victimDamageDealt': [{'basic': False,
    'magicDamage': 147,
    'name': 'Malzahar',
    'participantId': 7,
    'physicalDamage': 0,
    'spellName': 'malzaharq',
    'spellSlot': 0,
    'trueDamage': 0,
    'type': 'OTHER'},
   {'basic': False,
    'magicDamage': 196,
    'name': 'Malzahar',
    'participantId': 7,
    'physicalDamage': 0,
    'spellName': '',
    'spellSlot': -1,
    'trueDamage': 0,
    'type': 'OTHER'},
   {'basic': True,
    'magicDamage': 0,
    'name': 'Malzahar',
    'participantId': 7,
    'physicalDamage': 61,
    'spellName': 'malzaharbasicattack',
    'spellSlot': 64,
    'trueDamage': 0,
    'type': 'OTHER'},
   {'basic': False,
    'magicDamage': 120,
    'name': 'Malzahar',
    'participantId': 7,
    'physicalDamage': 0,
    'spellName': '',
    

In [40]:
k= list(map(lambda x:str(x['killerId']), kill_list))
d= list(map(lambda x:str(x['victimId']), kill_list))
# 어시스트는 1명 이상일 수 있기 때문에 리스트로 들어가 있다

In [41]:
k

['8', '4', '3', '3']

In [42]:
d

['1', '8', '9', '7']

In [50]:
assi_list = list(map(lambda x: str(x), kill_list[0]['assistingParticipantIds']))

In [51]:
','.join(assi_list)

'7'

In [52]:
def assi_calc(x):
    # 어시스트는 1명 이상일 수 있기 때문에 리스트로 들어가 있다
    try: # 어시스트가 없으면 에러뜨기 때문에 예외처리 해줌
        assi_list = list(map(lambda y: str(y), x['assistingParticipantIds']))
        return ','.join(assi_list)
    except:
        return ''

In [53]:
a = list(map(lambda x:assi_calc(x), kill_list))

In [54]:
a # ''가 나오는 이유는 어시스트 한 사람이 없기 때문

['7', '5', '', '']

In [55]:
k, d, a

(['8', '4', '3', '3'], ['1', '8', '9', '7'], ['7', '5', '', ''])

In [56]:
'|'.join(k)

'8|4|3|3'

In [57]:
'|'.join(d)

'1|8|9|7'

In [58]:
'|'.join(a)

'7|5||'

# 한게임의 들어가있는 킬/데스/어시를 다 모아서 아래의 형식으로 만드세요
- '|'.join(k)
- '|'.join(d)
- '|'.join(a)

In [59]:
# 내가 해본거 -> 시험공부때문에 하다 말았음
for i in tqdm(range(len(df.iloc[0].timeline['info']['frames']))):
    # prsnt(df.iloc[0].timeline['info']['frames'][i]['events'])
    kill_list = list(filter(lambda x: x if x['type'] == 'CHAMPION_KILL' else '' ,df.iloc[0].timeline['info']['frames'][i]['events']))
    k= list(map(lambda x:str(x['killerId']), kill_list))
    d= list(map(lambda x:str(x['victimId']), kill_list))
    a = list(map(lambda x:assi_calc(x), kill_list))
    k = '|'.join(k)
    d = '|'.join(d)
    a = '|'.join(a)
    print(a)


100%|███████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 15011.83it/s]




4|5

8|6,7|3
4|7,9|7,9
5|1,2|7
8

7,9|7,9|5|5
|||
5
9,10|9,10
2,5||

|9,10|9,10|2|10|1
1,5
5||
10|2,3,5|1,4,5|6|3,5|5

4,5
1,4,5|1,3,4,5|10
1,2,4,5
1,2,4,5
7|5||
6,8,10
1,3,5|1,4,5||9,10|3,5
5
4|8|1,3|3


In [60]:
# 강사님 코드 시작
tmp = df[:5].copy()

In [61]:
tmp.iloc[0].timeline['info']['frames']

[{'events': [{'realTimestamp': 1713971329726,
    'timestamp': 0,
    'type': 'PAUSE_END'}],
  'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
     'abilityPower': 0,
     'armor': 18,
     'armorPen': 0,
     'armorPenPercent': 0,
     'attackDamage': 25,
     'attackSpeed': 100,
     'bonusArmorPenPercent': 0,
     'bonusMagicPenPercent': 0,
     'ccReduction': 0,
     'cooldownReduction': 0,
     'health': 580,
     'healthMax': 580,
     'healthRegen': 0,
     'lifesteal': 0,
     'magicPen': 0,
     'magicPenPercent': 0,
     'magicResist': 30,
     'movementSpeed': 335,
     'omnivamp': 0,
     'physicalVamp': 0,
     'power': 375,
     'powerMax': 375,
     'powerRegen': 0,
     'spellVamp': 0},
    'currentGold': 500,
    'damageStats': {'magicDamageDone': 0,
     'magicDamageDoneToChampions': 0,
     'magicDamageTaken': 0,
     'physicalDamageDone': 0,
     'physicalDamageDoneToChampions': 0,
     'physicalDamageTaken': 0,
     'totalDamageDone': 0,
     'tota

In [62]:
tmp_map = list(map(lambda x: x['events'], tmp.iloc[0].timeline['info']['frames']))

In [63]:
tmp_map # 이중리스트임 -> 하나의 리스트로 바꿔주자
# 이중리스트에서 요소를 하나씩 뺸 후, 뺀 요소에서 또 요소를 빼서 하나의 리스트로

[[{'realTimestamp': 1713971329726, 'timestamp': 0, 'type': 'PAUSE_END'}],
 [{'itemId': 3865,
   'participantId': 5,
   'timestamp': 1743,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 5,
   'timestamp': 1743,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 5,
   'timestamp': 1743,
   'type': 'ITEM_PURCHASED'},
  {'levelUpType': 'NORMAL',
   'participantId': 7,
   'skillSlot': 3,
   'timestamp': 1910,
   'type': 'SKILL_LEVEL_UP'},
  {'itemId': 1056,
   'participantId': 6,
   'timestamp': 1943,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 6,
   'timestamp': 2209,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 3340,
   'participantId': 5,
   'timestamp': 2310,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 6,
   'timestamp': 2344,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 3865,
   'participantId': 10,
   'timestamp': 2712,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 10,
   'time

In [64]:
tmp_map = [element for array in tmp_map for element in array]

In [65]:
tmp_map

[{'realTimestamp': 1713971329726, 'timestamp': 0, 'type': 'PAUSE_END'},
 {'itemId': 3865,
  'participantId': 5,
  'timestamp': 1743,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 5,
  'timestamp': 1743,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 5,
  'timestamp': 1743,
  'type': 'ITEM_PURCHASED'},
 {'levelUpType': 'NORMAL',
  'participantId': 7,
  'skillSlot': 3,
  'timestamp': 1910,
  'type': 'SKILL_LEVEL_UP'},
 {'itemId': 1056,
  'participantId': 6,
  'timestamp': 1943,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 6,
  'timestamp': 2209,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3340,
  'participantId': 5,
  'timestamp': 2310,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 6,
  'timestamp': 2344,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3865,
  'participantId': 10,
  'timestamp': 2712,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 10,
  'timestamp': 2712,
  'type': 'ITEM_PURCHASED'},

In [66]:
kill_log = list(filter(lambda x: x['type'] == 'CHAMPION_KILL', tmp_map))

In [67]:
kill_log

[{'assistingParticipantIds': [4],
  'bounty': 400,
  'killStreakLength': 0,
  'killerId': 5,
  'position': {'x': 12478, 'y': 1451},
  'shutdownBounty': 0,
  'timestamp': 129796,
  'type': 'CHAMPION_KILL',
  'victimDamageDealt': [{'basic': True,
    'magicDamage': 0,
    'name': 'Rakan',
    'participantId': 4,
    'physicalDamage': 98,
    'spellName': 'rakanbasicattackshort',
    'spellSlot': 49,
    'trueDamage': 0,
    'type': 'OTHER'},
   {'basic': True,
    'magicDamage': 0,
    'name': 'Rakan',
    'participantId': 4,
    'physicalDamage': 49,
    'spellName': 'rakanbasicattackshort2',
    'spellSlot': 49,
    'trueDamage': 0,
    'type': 'OTHER'},
   {'basic': True,
    'magicDamage': 0,
    'name': 'Rakan',
    'participantId': 5,
    'physicalDamage': 36,
    'spellName': 'rakanbasicattack',
    'spellSlot': 49,
    'trueDamage': 0,
    'type': 'OTHER'},
   {'basic': False,
    'magicDamage': 51,
    'name': 'Rakan',
    'participantId': 5,
    'physicalDamage': 0,
    'spellN

In [68]:
def assi_calc(x): # 아까 만든 함수인데 이렇게 한번에 쓸 수도 있음
    try:
        return ','.join(list(map(lambda y: str(y), x['assistingParticipantIds'])))
    except:
        return ''

In [69]:
k = list(map(lambda x: str(x['killerId']), kill_log))
d = list(map(lambda x: str(x['victimId']), kill_log))
a = list(map(lambda x: assi_calc(x), kill_log))

In [70]:
# k,d,a를 데이터프레임으로 만들어보자
col_list = ['matchId', 'gameDuration', 'bans', 'killerId', 'victimId', 'assistId']

In [71]:
df_create = []
for i in range(len(tmp)):
    matchId = tmp.iloc[i].match_id
    match = tmp.iloc[i].matches['info']
    timeline = tmp.iloc[i].timeline['info']
    for j in range(10):
        lst_tmp = []
        lst_tmp.append(matchId)
        lst_tmp.append(match['gameDuration'])

        blue_ban = list(map(lambda x: str(x['championId']), match['teams'][0]['bans']))
        red_ban = list(map(lambda x: str(x['championId']), match['teams'][1]['bans']))
        ban_list = list(set(blue_ban+red_ban)) # 중복된 값이 있을 수도 있으니 set을 사용해서 중복제거
        lst_tmp.append('|'.join(ban_list))

        kill_log = list(map(lambda x: list(filter(lambda z: z['type'] == 'CHAMPION_KILL' , x['events'])) , timeline['frames']))
        kill_log = [element for array in kill_log for element in array]
        k = list(map(lambda x: str(x['killerId']), kill_log))
        d = list(map(lambda x: str(x['victimId']), kill_log))
        a = list(map(lambda x: assi_calc(x), kill_log))
        lst_tmp.append('|'.join(k))
        lst_tmp.append('|'.join(d))
        lst_tmp.append('|'.join(a))
        df_create.append(lst_tmp)

df = pd.DataFrame(df_create, columns=col_list)
        

In [72]:
df

,matchId,gameDuration,bans,killerId,victimId,assistId
0,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
1,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
2,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
3,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
4,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
5,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
6,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
7,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
8,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."
9,KR_7042551213,1739,266|268|56|154|53|111|122|777|35,5|4|7|8|2|5|8|8|4|3|6|9|6|10|10|4|4|4|7|6|1|3|...,10|9|2|1|7|9|5|4|9|7|1|4|1|4|5|10|9|7|2|1|6|8|...,"4|5|8|6,7|3|4|7,9|7,9|5|1,2|7|8||7,9|7,9|5|5||..."


# Tower log

- type => BUILDING_KILL
- first tower를 부신 사람을 알고싶음
- 라인전에서 어떤 팀이 해당 라인의 타워를 빨리 부셨나의 여부
- 타워가 부서진 시간도 알고싶음

In [73]:
tmp.iloc[0].timeline['info']['frames'] 

[{'events': [{'realTimestamp': 1713971329726,
    'timestamp': 0,
    'type': 'PAUSE_END'}],
  'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
     'abilityPower': 0,
     'armor': 18,
     'armorPen': 0,
     'armorPenPercent': 0,
     'attackDamage': 25,
     'attackSpeed': 100,
     'bonusArmorPenPercent': 0,
     'bonusMagicPenPercent': 0,
     'ccReduction': 0,
     'cooldownReduction': 0,
     'health': 580,
     'healthMax': 580,
     'healthRegen': 0,
     'lifesteal': 0,
     'magicPen': 0,
     'magicPenPercent': 0,
     'magicResist': 30,
     'movementSpeed': 335,
     'omnivamp': 0,
     'physicalVamp': 0,
     'power': 375,
     'powerMax': 375,
     'powerRegen': 0,
     'spellVamp': 0},
    'currentGold': 500,
    'damageStats': {'magicDamageDone': 0,
     'magicDamageDoneToChampions': 0,
     'magicDamageTaken': 0,
     'physicalDamageDone': 0,
     'physicalDamageDoneToChampions': 0,
     'physicalDamageTaken': 0,
     'totalDamageDone': 0,
     'tota

In [74]:
event_list = list(map(lambda x: x['events'], tmp.iloc[0].timeline['info']['frames']))

In [75]:
event_list = [element for array in event_list for element in array]

In [76]:
tower_log = list(filter(lambda x: x['type']=='BUILDING_KILL', event_list))

In [77]:
tower_log 

[{'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 3,
  'laneType': 'TOP_LANE',
  'position': {'x': 4318, 'y': 13875},
  'teamId': 200,
  'timestamp': 850419,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 4,
  'laneType': 'BOT_LANE',
  'position': {'x': 13866, 'y': 4505},
  'teamId': 200,
  'timestamp': 1011147,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [10],
  'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 7,
  'laneType': 'MID_LANE',
  'position': {'x': 5846, 'y': 6396},
  'teamId': 100,
  'timestamp': 1083346,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [1],
  'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 4,
  'laneType': 'TOP_LANE',
  'position': {'x': 7943, 'y': 13411},
  'teamId': 200,
  'timestamp': 1149449,
  'towerType': 'INNER_TURRET',
  'type': 'BUILDING_KILL'},
 

- laneTower ==> 'laneType'== teamPosition (똑같지않고 특정 방법을 통해서 인식) & 'teamId' == teamId ( 첫번째타워 꺠진거만 데이터 입력)
- firstDT -> 최대 한개만 1이 들어감
- laneTower - > 타워가 깨졌을 시 1이 들어감 (top, mid, bot에만 데이터가 1이 들어가야함)
- laneTowerTime -> 깨진 시간(timestamp)를 넣으면 됨
- 데이터프레임 df_create 추가

### col_lst = ['matchId','gameDuration','gameVersion', 'participantId', 'teamPosition','teamId', 'firstDT','laneTower','LaneTowerTime' ,'bans','killerId','victimId','assistId']

In [ ]:
Postion    participantId  TEAM    firstDT     laneTower     laneTowerTime
   top          1           100   0               1            342038
   mid          2                 0               1            904783
   jungle       3                 0               0            0
   bot          4                 1               1            828853
   sup          5                 0               0            0
   jungle       6           200   0               0            0
   mid          7                 0               1            0
   top          8                 0               1            843507
   bot          9                 0               0            0
   sup          10                0               0            0

- first_DT - > 해당 라인의 타워가 게임에서 가장 먼저 부셔졌을 경우에 1 아닐경우에 0

- laneTower -> 해당 라인의 타워가 게임에서 한개라도 부셔졌을경우 1 아닐경우 0

- laneTowerTime -> 가장 먼저 부셔진타워의 timestamp 작성

In [79]:
tower_log[0]['killerId'] # 첫번째 타워를 부신 사람

3

In [89]:
db_open()
test = sql_execute('select * from lol_datas')
db_close()

오라클 정상적으로 닫힘


C:\Users\user\게임아이 수업자료\my_utils.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=db)


In [96]:
df3 = get_rawdata('SILVER')

get SummonerId


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.42it/s]


get PUUID......


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.57it/s]


get match id.....


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:08<00:00,  1.49it/s]


get maches&timeline....


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:38<00:00,  1.62s/it]


In [100]:
df3.matches.iloc[0]

{'metadata': {'dataVersion': '2',
  'matchId': 'KR_6931716674',
  'participants': ['U7nvCDSpZVXER3Ymj-NFSAVGIjhR-26gGWKpM9lmoNn-cIqb0V1sAJjl7c7Q7mrwoDqctPWo4AN9fQ',
   'L3ogwP3Zygb1AaE4DGRC38cUIRsGXnELfprDR4vWQDMGx_bQQ6Al_BCDWiw6w9u58IbUOX2TwWfaHA',
   '5YctK8JA09TSbDbE4TKqlRA8LCu5CiBBxdgqVLVViofvJd09KeZDEvRaoSHZYsxpH3iXXQwgQ8_7Hw',
   'ul7veQps4PNasas2uItuoYtugOw79Ql6FRHCh7WyGU3dzK7sIkIPOLdVO_QyQoV0eABTBUSKs1CQhg',
   'kbkmJ_ZHM4SQzOkbORPpivx55wWHW6ACT0YJaQsdLss_W7jwHOQNGQvqdWb9sprNbDg4QZrXhbPBtA',
   'belXsrHLnxuDV1MPSS1Qs98TtEVwno_kK-v56hdznJAbHF_HOGtSp2ZFELfIj7i7ZNS4SUpRmtkUfg',
   'RyOe_B60iwoHr0a6EORxgu_Eej081M7Rq6I5cMR4WcHzKSMOhTmDLvgrJmuJeXaaP0-1VWZ5rpTqaQ',
   'NSQjjf5zxqlnh5o-U4zrSAMEfV1J_3EvQcfd8xFmtg-w1Frh3yV1GuizBISpCBf1kCLW210WcXmR-g',
   'Xuim0y95gJHcaVdS4T5eJOOkWwGTHo-mgfRA_Ybvu5ypf_R7CdWfgxz_eTYLNNA5THO2ZtOxfWcd2w',
   'mN_fdN_y6u77IHsoPdWAulY__N3I4TvsZuFGzOR6HI8Dj9KenTMZmxgN7tQIDvxfETGA1dG9-eE_kw']},
 'info': {'gameCreation': 1706966935764,
  'gameDuration': 1756,
  '